In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Schritt 1: Definieren Sie Pydantic-Modelle für strukturierte Ausgaben

Diese Modelle definieren das **Schema**, das von den Agenten zurückgegeben wird. Die Verwendung von `response_format` mit Pydantic gewährleistet:
- ✅ Typensichere Datenextraktion
- ✅ Automatische Validierung
- ✅ Keine Parsing-Fehler bei Freitextantworten
- ✅ Einfache bedingte Weiterleitung basierend auf Feldern


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Schritt 2: Erstellen des Hotelbuchungs-Tools

Dieses Tool wird vom **availability_agent** aufgerufen, um zu prüfen, ob Zimmer verfügbar sind. Wir verwenden den `@ai_function`-Dekorator, um:
- Eine Python-Funktion in ein KI-aufrufbares Tool umzuwandeln
- Automatisch ein JSON-Schema für das LLM zu generieren
- Die Parametervalidierung zu übernehmen
- Die automatische Ausführung durch Agenten zu ermöglichen

Für diese Demo:
- **Stockholm, Seattle, Tokio, London, Amsterdam** → Zimmer verfügbar ✅
- **Alle anderen Städte** → Keine Zimmer ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Schritt 3: Bedingungsfunktionen für das Routing definieren

Diese Funktionen prüfen die Antwort des Agenten und bestimmen, welchen Pfad im Workflow eingeschlagen wird.

**Wichtiges Muster:**
1. Überprüfen, ob die Nachricht `AgentExecutorResponse` ist
2. Das strukturierte Ergebnis (Pydantic-Modell) analysieren
3. `True` oder `False` zurückgeben, um das Routing zu steuern

Der Workflow wird diese Bedingungen an **Knotenpunkten** auswerten, um zu entscheiden, welcher Executor als nächstes aufgerufen wird.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## Schritt 4: Erstellen eines benutzerdefinierten Anzeige-Executors

Executors sind Workflow-Komponenten, die Transformationen oder Nebeneffekte ausführen. Wir verwenden den `@executor`-Dekorator, um einen benutzerdefinierten Executor zu erstellen, der das Endergebnis anzeigt.

**Wichtige Konzepte:**
- `@executor(id="...")` - Registriert eine Funktion als Workflow-Executor
- `WorkflowContext[Never, str]` - Typ-Hinweise für Eingabe/Ausgabe
- `ctx.yield_output(...)` - Gibt das endgültige Workflow-Ergebnis zurück


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## Schritt 5: Laden von Umgebungsvariablen

Konfigurieren Sie den LLM-Client. Dieses Beispiel funktioniert mit:
- **GitHub-Modelle** (Kostenlose Stufe mit GitHub-Token)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## Schritt 6: Erstellen von KI-Agenten mit strukturierten Ausgaben

Wir erstellen **drei spezialisierte Agenten**, die jeweils in einem `AgentExecutor` eingebettet sind:

1. **availability_agent** - Überprüft die Verfügbarkeit von Hotels mithilfe des Tools
2. **alternative_agent** - Schlägt alternative Städte vor (wenn keine Zimmer verfügbar sind)
3. **booking_agent** - Ermutigt zur Buchung (wenn Zimmer verfügbar sind)

**Wichtige Merkmale:**
- `tools=[hotel_booking]` - Stellt dem Agenten das Tool zur Verfügung
- `response_format=PydanticModel` - Erzwingt eine strukturierte JSON-Ausgabe
- `AgentExecutor(..., id="...")` - Betten den Agenten für die Nutzung im Workflow ein


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Schritt 7: Erstellen des Workflows mit bedingten Kanten

Jetzt verwenden wir `WorkflowBuilder`, um den Graphen mit bedingter Weiterleitung zu erstellen:

**Workflow-Struktur:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**Wichtige Methoden:**
- `.set_start_executor(...)` - Legt den Einstiegspunkt fest
- `.add_edge(from, to, condition=...)` - Fügt eine bedingte Kante hinzu
- `.build()` - Finalisiert den Workflow


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## Schritt 8: Testfall 1 ausführen - Stadt OHNE Verfügbarkeit (Paris)

Testen wir den **Keine Verfügbarkeit**-Pfad, indem wir Hotels in Paris anfragen (das in unserer Simulation keine Zimmer hat).


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## Schritt 9: Testfall 2 ausführen - Stadt MIT Verfügbarkeit (Stockholm)

Lassen Sie uns nun den **Verfügbarkeits**-Pfad testen, indem wir Hotels in Stockholm anfragen (die in unserer Simulation Zimmer haben).


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## Wichtige Erkenntnisse und nächste Schritte

### ✅ Was Sie gelernt haben:

1. **WorkflowBuilder-Muster**
   - Verwenden Sie `.set_start_executor()`, um den Einstiegspunkt zu definieren
   - Nutzen Sie `.add_edge(from, to, condition=...)` für bedingte Weiterleitungen
   - Rufen Sie `.build()` auf, um den Workflow abzuschließen

2. **Bedingte Weiterleitung**
   - Bedingungsfunktionen prüfen `AgentExecutorResponse`
   - Strukturierte Ausgaben analysieren, um Weiterleitungsentscheidungen zu treffen
   - Geben Sie `True` zurück, um eine Verbindung zu aktivieren, `False`, um sie zu überspringen

3. **Tool-Integration**
   - Verwenden Sie `@ai_function`, um Python-Funktionen in KI-Tools umzuwandeln
   - Agenten rufen Tools automatisch auf, wenn sie benötigt werden
   - Tools geben JSON zurück, das von Agenten analysiert werden kann

4. **Strukturierte Ausgaben**
   - Nutzen Sie Pydantic-Modelle für eine typensichere Datenextraktion
   - Setzen Sie `response_format=MyModel`, wenn Sie Agenten erstellen
   - Analysieren Sie Antworten mit `Model.model_validate_json()`

5. **Benutzerdefinierte Executors**
   - Verwenden Sie `@executor(id="...")`, um Workflow-Komponenten zu erstellen
   - Executors können Daten transformieren oder Nebeneffekte ausführen
   - Nutzen Sie `ctx.yield_output()`, um Workflow-Ergebnisse zu erzeugen

### 🚀 Anwendungen in der Praxis:

- **Reisebuchung**: Verfügbarkeit prüfen, Alternativen vorschlagen, Optionen vergleichen
- **Kundendienst**: Weiterleitung basierend auf Problemtyp, Stimmung, Priorität
- **E-Commerce**: Lagerbestand prüfen, Alternativen vorschlagen, Bestellungen bearbeiten
- **Inhaltsmoderation**: Weiterleitung basierend auf Toxizitätswerten, Nutzerkennzeichnungen
- **Genehmigungs-Workflows**: Weiterleitung basierend auf Betrag, Nutzerrolle, Risikostufe
- **Mehrstufige Verarbeitung**: Weiterleitung basierend auf Datenqualität, Vollständigkeit

### 📚 Nächste Schritte:

- Komplexere Bedingungen hinzufügen (mehrere Kriterien)
- Schleifen mit Workflow-Zustandsverwaltung implementieren
- Unter-Workflows für wiederverwendbare Komponenten hinzufügen
- Integration mit echten APIs (Hotelbuchung, Lagerbestandsysteme)
- Fehlerbehandlung und alternative Pfade hinzufügen
- Workflows mit den integrierten Visualisierungstools darstellen



---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
